# Preparations

In [1]:
#!pip install -r requirements.txt
import requests
from urllib.parse import urlencode
import googlemaps
import os
from dotenv import load_dotenv
import time
import pandas as pd

In [2]:
# import the api_key from the api_key.py file
from api_key import api_key

# City boundaries & grid

In [ ]:
# Function to get the city boundaries
def get_city_boundaries(city_name):
    # Initialize the Google Maps client
    gmaps = googlemaps.Client(key=api_key) # googlemaps package
    
    # Get the city boundaries
    geocode_result = gmaps.geocode(city_name)

    # save the boundaries
    low_lat = geocode_result[0]['geometry']['bounds']['southwest']['lat']
    low_long = geocode_result[0]['geometry']['bounds']['southwest']['lng']
    high_lat = geocode_result[0]['geometry']['bounds']['northeast']['lat']
    high_long = geocode_result[0]['geometry']['bounds']['northeast']['lng']

    
    return low_lat, low_long, high_lat, high_long

In [16]:
# Function to divide the city into a grid
def divide_area_in_grid(boundary, step_size = 0.01):
    low_lat, low_long, high_lat, high_long = boundary
    grid = []
    lat = low_lat
    while lat < high_lat:
        long = low_long
        while long < high_long:
            cell = (lat, long, min(lat + step_size, high_lat), min(long + step_size, high_long))
            grid.append(cell)
            long += step_size
        lat += step_size
    return grid    

# Get places data

In [ ]:
# all fields that are interesting for us
search_fields = [
    # Fields that trigger the Text Search (ID Only) SKU
    "places.displayName", "places.attributions", "places.id", "nextPageToken",

    # Fields that trigger the Text Search (Basic) SKU
    "places.accessibilityOptions",
    "places.businessStatus", "places.containingPlaces", "places.displayName", 
    "places.formattedAddress", "places.googleMapsLinks", 
    "places.location", "places.primaryType", "places.primaryTypeDisplayName", 
    "places.pureServiceAreaBusiness", "places.subDestinations", 
    "places.types", "places.utcOffsetMinutes", "places.viewport",

    # Fields that trigger the Text Search (Advanced) SKU
    "places.internationalPhoneNumber", "places.nationalPhoneNumber", "places.priceLevel", 
    "places.priceRange", "places.rating", "places.regularOpeningHours", 
    "places.userRatingCount", "places.websiteUri",

    # Fields that trigger the Text Search (Preferred) SKU
    "places.allowsDogs", "places.curbsidePickup", "places.delivery", "places.dineIn", 
    "places.editorialSummary", "places.evChargeOptions", "places.fuelOptions", 
    "places.goodForChildren", "places.goodForGroups", "places.goodForWatchingSports", 
    "places.liveMusic", "places.menuForChildren", "places.parkingOptions", 
    "places.paymentOptions", "places.outdoorSeating", "places.reservable", "places.restroom", 
    "places.reviews", "places.servesBeer", 
    "places.servesBreakfast", "places.servesBrunch", "places.servesCocktails", 
    "places.servesCoffee", "places.servesDessert", "places.servesDinner", 
    "places.servesLunch", "places.servesVegetarianFood", "places.servesWine", 
    "places.takeout"
]

In [ ]:
def get_places_data(boundary):
    low_lat, low_long, high_lat, high_long = boundary

    # Base URL
    base_url = "https://places.googleapis.com/v1/places:searchText"
    
    # Headers
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": ",".join(search_fields) 
    }

    results = [] # List to store the results
    page_token = None  # Initialize the page token to None

    while True:
        # JSON request body
        body = {
            "includedType": "restaurant",  # Restrict to restaurants
            "strictTypeFiltering": True,  # Only return results of the specified type
            "textQuery": "restaurant",
            "pageSize": 20,  # max results per page
            "pageToken": page_token,  # Next page token, if any
            "languageCode": "en",  # Language for results
            "locationRestriction": {
                "rectangle": {
                    "low": {"latitude": low_lat, "longitude": low_long},
                    "high": {"latitude": high_lat, "longitude": high_long}
                }}
        }

        # Send the POST request to the API
        response = requests.post(base_url, headers=headers, json=body)
        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()

            # Check if "places" key is present in the response
            if "places" in data:
                results.extend(data["places"])
            else: break # Break the loop if no results are available

            # Check if there is a next page token
            if "nextPageToken" in data:
                page_token = data["nextPageToken"]
            else:
                break
        else:
            print("Error:", response.status_code, "Message:", response.text)
            break
    
    return results

# Get the data from the grid

In [ ]:
def get_places_data_in_grid(grid):
    all_results = []

    for cell in grid:
        results = get_places_data(cell)
        time.sleep(2)

        if len(results) != 60:
            all_results.extend(results)

        # If API returns full results, recursively split the cell
        if len(results) == 60:
            print(f"Overcrowded cell detected: {cell}")
            low_lat, low_long, high_lat, high_long = cell
            mid_lat = (low_lat + high_lat) / 2
            mid_long = (low_long + high_long) / 2
            sub_cells = [
                (low_lat, low_long, mid_lat, mid_long),  # Bottom-left
                (low_lat, mid_long, mid_lat, high_long),  # Bottom-right
                (mid_lat, low_long, high_lat, mid_long),  # Top-left
                (mid_lat, mid_long, high_lat, high_long)  # Top-right
            ]

            # Recursive call for each sub-cell
            subdivided_results = get_places_data_in_grid(sub_cells)
            all_results.extend(subdivided_results)  # Append all subdivided results
    
    return all_results